In [52]:
import numpy as np
import cv2

cap = cv2.VideoCapture('video.mp4')

frame_count = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT))
label_result = [-1] * frame_count
fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)

internal = 1000 / fps
internal_ary = [int(internal), int(internal * 2), int(internal * 4)]
internal_idx = 0
frame_idx = 0
frame_num = 20

def draw_text(img, label, set_label, internel_idx):
    text = ["none", "normal", "paging", "shake", "else"]
    speed_text = ["1X", "0.5X", "0.25X"]
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, str(frame_idx) + '/' + str(frame_count), (10, height - 20), font, 1, (0,0,255), 2)
    cv2.putText(img, "Current set: " + text[set_label + 1], (width - 400, 30), font, 1, (0, 0, 255), 2)
    cv2.putText(img, speed_text[internal_idx], (10, 30), font, 1, (0, 0, 255), 2)
    if label != -1:
        cv2.putText(img, text[label + 1], (width - 200, height - 20), font, 1, (0, 0, 255), 2)
    

current_label = -1
    
while cap.isOpened():
    ret, frame = cap.read()
    if ret != True:
        break
    frame_idx = frame_idx + 1
    draw_text(frame, label_result[frame_idx - 1], current_label, internal_idx)
    cv2.imshow('frame',frame)
    if current_label != -1:
        label_result[frame_idx - 1] = current_label
    k = cv2.waitKey(internal_ary[internal_idx])
    if k & 0xFF == ord('q'):
        break
    if k & 0xFF == ord('t'):
        r = cv2.waitKey()
        while r & 0xFF != ord('t'):
            if r & 0xFF == ord('h'):
                # go back
                frame_idx = np.max([0, frame_idx - frame_num])
                cap.set(1, frame_idx)
                frame_idx = frame_idx + 1
                ret, frame = cap.read()
                current_label = -1
                draw_text(frame, label_result[frame_idx - 1], current_label, internal_idx)
                cv2.imshow('frame', frame)
            if r & 0xFF == ord('l'):
                # go forward
                temp = frame_idx
                frame_idx = np.min([frame_count, frame_idx + frame_num])
                cap.set(1, frame_idx)
                frame_idx = frame_idx + 1
                ret, frame = cap.read()
                draw_text(frame, label_result[frame_idx - 1], current_label, internal_idx)
                cv2.imshow('frame', frame)
                for x in range(temp, frame_idx):
                    label_result[x] = current_label
            r = cv2.waitKey()
    if k & 0xFF == ord('h'):
        # go back
        frame_idx = np.max([0, frame_idx - frame_num])
        cap.set(1, frame_idx)
        frame_idx = frame_idx + 1
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        current_label = -1
    if k & 0xFF == ord('l'):
        # go forward
        temp = frame_idx
        frame_idx = np.min([frame_count, frame_idx + frame_num])
        cap.set(1, frame_idx)
        frame_idx = frame_idx + 1
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        for x in range(temp, frame_idx):
            label_result[x] = current_label
    if k & 0xFF == ord('j'):
        internal_idx = np.max([internal_idx - 1, 0])
    if k & 0xFF == ord('k'):
        internal_idx = np.min([internal_idx + 1, 2])
    if k & 0xFF == ord('0'):
        current_label = 0
    if k & 0xFF == ord('1'):
        current_label = 1
    if k & 0xFF == ord('2'):
        current_label = 2
    if k & 0xFF == ord('3'):
        current_label = 3
    if k & 0xFF == ord('c'):
        current_label = -1

cap.release()
cv2.destroyAllWindows()

# finally, save the label_result